In [ ]:
import org.bdgenomics.adam.rdd.ADAMContext._
import comp.bio.aging.playground.extensions._
import org.apache.spark.sql._
import org.apache.spark.sql.SparkSession
import frameless.functions.aggregate
import frameless.TypedDataset
import frameless.syntax._

implicit val session = SparkSession
  .builder()
  .appName("Spark SQL basic example")
  .config("spark.some.config.option", "some-value")
  .getOrCreate()


import org.bdgenomics.adam.rdd.ADAMContext._
import comp.bio.aging.playground.extensions._
import org.apache.spark.sql._
import org.apache.spark.sql.SparkSession
import frameless.functions.aggregate
import frameless.TypedDataset
import frameless.syntax._
session: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@792ff40d


In [ ]:
val base = "hdfs://namenode/pipelines"
val ml = s"${base}/ml"
val genAgePath = s"${ml}/genage_LAGs_list.xlsx"
val vladFilePath = s"${ml}/ML validation (11.12.2017).xlsx"
  

base: String = hdfs://namenode/pipelines
ml: String = hdfs://namenode/pipelines/ml
genAgePath: String = hdfs://namenode/pipelines/ml/genage_LAGs_list.xlsx
vladFilePath: String = hdfs://namenode/pipelines/ml/ML validation (11.12.2017).xlsx


In [ ]:
def openSpreadsheet(location: String, sheet: String, emptyAsNulls: Boolean = false, colorColumns: Boolean = false)
                   (implicit sessions: SparkSession): DataFrame = {
  sessions.read
    .format("com.crealytics.spark.excel")
    .option("sheetName", sheet) // Required
    .option("useHeader", "true") // Required
    .option("treatEmptyValuesAsNulls", emptyAsNulls.toString) // Optional, default: true
    .option("inferSchema", "true") // Optional, default: false
    .option("addColorColumns", colorColumns.toString) // Optional, default: false
    .load(location)
}

openSpreadsheet: (location: String, sheet: String, emptyAsNulls: Boolean, colorColumns: Boolean)(implicit sessions: org.apache.spark.sql.SparkSession)org.apache.spark.sql.DataFrame


In [ ]:
val genAgeHuman = openSpreadsheet(genAgePath, "genage_human")
  .withColumnRenamed("entrez gene id", "entrez").cache
genAgeHuman

genAgeHuman: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [GenAge ID: double, symbol: string ... 14 more fields]
res4: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [GenAge ID: double, symbol: string ... 14 more fields]


In [ ]:
val genAgeModel = openSpreadsheet(genAgePath, "genage_models").withColumnRenamed("entrez gene id", "entrez").cache
genAgeModel

In [ ]:
import frameless.TypedDataset
import frameless.syntax._
import session.implicits._

import org.apache.spark.sql.functions._

val toDouble = udf[Double, String](s => if(s!=null) s.replace(",","").toDouble else 0.0)

val vladGenes = Seq("WRN", "LMNA", "FOXO1", "APOE", "Gdf15", "GHR", "Ghrh", "IGF1", "Pou1f1", "age-1", "daf-2", "Tor", "osm-5", "unc-13")
def openVladSheet(sheet: String) = {
  val df = openSpreadsheet(vladFilePath, sheet)
  df
    //.withColumn("Entrez", toInt(df("Entrez")))
    .withColumn("Score", toDouble(df("Score")))
    //.withColumn("TaxID", toInt(df("TaxID")))
    .withColumnRenamed("Effect on CS (CellAge)", "CellAge")
    //.withColumnRenamed("C6", "GenAge")
    .withColumn("predicted_for", lit(sheet))
    .select("predicted_for", "ORD", "Entrez",   "TaxID", "Score", "Symbol", "CellAge", "Comment")  
}
val predictions = vladGenes.map(g=>openVladSheet(g)).reduce((a,b)=> a.union(b)).cache
predictions


import frameless.TypedDataset
import frameless.syntax._
import session.implicits._
import org.apache.spark.sql.functions._
toDouble: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function1>,DoubleType,Some(List(StringType)))
vladGenes: Seq[String] = List(WRN, LMNA, FOXO1, APOE, Gdf15, GHR, Ghrh, IGF1, Pou1f1, age-1, daf-2, Tor, osm-5, unc-13)
openVladSheet: (sheet: String)org.apache.spark.sql.DataFrame
predictions: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [predicted_for: string, ORD: string ... 6 more fields]
res8: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [predicted_for: string, ORD: string ... 6 more fields]


In [ ]:
predictions.show(100)

+-------------+---+------------+-------+---------+---------+--------------------+--------------------+
|predicted_for|ORD|      Entrez|  TaxID|    Score|   Symbol|             CellAge|             Comment|
+-------------+---+------------+-------+---------+---------+--------------------+--------------------+
|          WRN| 0.|      7486.0| 9606.0|1000000.0|      WRN|Inhibits;WRN expr...|Werner progeroid ...|
|          WRN| 1.|     55183.0| 9606.0| 0.883706|     RIF1|                null|                null|
|          WRN| 2.|      5981.0| 9606.0| 0.877486|     RFC1|                null|                null|
|          WRN| 3.|     56916.0| 9606.0| 0.856532| SMARCAD1|                null|                null|
|          WRN| 4.|    257218.0| 9606.0| 0.856138|    SHPRH|                null|                null|
|          WRN| 5.|     84146.0| 9606.0| 0.856087|   ZNF644|                null|                null|
|          WRN| 6.|       545.0| 9606.0| 0.853317|      ATR|             

In [ ]:
genAgeModel

res12: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [GenAge ID: double, symbol: string ... 6 more fields]


In [ ]:
val modeled = predictions
  .join(genAgeModel, genAgeModel("entrez") === predictions("Entrez"))
  .drop(genAgeModel("entrez"))  
  .drop(genAgeModel("symbol"))  
  .cache
modeled

modeled: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [predicted_for: string, ORD: string ... 12 more fields]
res14: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [predicted_for: string, ORD: string ... 12 more fields]


In [ ]:
val humaned  = predictions
  .join(genAgeHuman, predictions("Entrez") === genAgeHuman("entrez"))
  .drop(genAgeHuman("entrez"))  
  .drop(genAgeHuman("symbol"))  
  .cache
humaned

humaned: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [predicted_for: string, ORD: string ... 20 more fields]
res16: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [predicted_for: string, ORD: string ... 20 more fields]


In [ ]:
val predictedIds = 
    modeled.select("Entrez").map(_.getDouble(0)).collect() ++ humaned.select("Entrez").map(_.getDouble(0)).collect()
val notInGenAge = predictions.filter( r=> !predictedIds.contains(r.getAs[Double]("Entrez")))
notInGenAge

predictedIds: Array[Double] = Array(23886.0, 14600.0, 14601.0, 16000.0, 18736.0, 174762.0, 175410.0, 180982.0, 178104.0, 174462.0, 176065.0, 175559.0, 181055.0, 178113.0, 172167.0, 177770.0, 173960.0, 180585.0, 189055.0, 180401.0, 179666.0, 172497.0, 179412.0, 7486.0, 545.0, 7155.0, 4000.0, 2308.0, 4303.0, 2309.0, 1385.0, 7329.0, 3066.0, 5468.0, 23411.0, 348.0, 2.0, 5663.0, 6648.0, 2690.0, 6464.0, 5295.0, 3717.0, 5781.0, 3643.0, 3667.0, 5747.0, 25.0, 5159.0, 3480.0, 8835.0, 2692.0, 3479.0, 3481.0, 3486.0, 5054.0, 5449.0, 5468.0, 10891.0, 5469.0, 5465.0, 3725.0, 1051.0)
notInGenAge: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [predicted_for: string, ORD: string ... 6 more fields]
res18: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [predicted_for: string, ORD: str...

In [ ]:
modeled.coalesce(1).write.option("sep", "\t").csv(ml + "/predictions_models.tsv")
humaned.coalesce(1).write.option("sep", "\t").csv(ml + "/predictions_human.tsv")
notInGenAge.coalesce(1).write.option("sep", "\t").csv(ml + "/predictions_notfound.tsv")


In [ ]:
(predictions.count, modeled.count, humaned.count, notInGenAge.count)

res140: (Long, Long, Long, Long) = (437,23,40,374)


(437,23,40,374)